In [1]:
from custom_dataset.segdataset import InstanceSegmentationDataset
from torch.utils.data import DataLoader
import json
import torch

In [2]:
from transformers import SegformerImageProcessor

root_dir = '/disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe'
#----------------------
# reduce_labels should be False !! 

image_processor = SegformerImageProcessor(reduce_labels=False)

train_dataset = InstanceSegmentationDataset(root_dir=root_dir, image_processor=image_processor)
valid_dataset = InstanceSegmentationDataset(root_dir=root_dir, image_processor=image_processor, train=False)

/home/eric/anaconda3/envs/trex/lib/python3.9/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [3]:
image_processor

SegformerImageProcessor {
  "do_normalize": true,
  "do_reduce_labels": false,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "SegformerImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 512,
    "width": 512
  }
}

In [4]:
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=2)

In [5]:
json_file = '/disk3/eric/dataset/VISION_SOFS/WEAPON_4/segmentation_pipe/train/_annotations.coco.json'
with open(json_file, 'r') as f:
    data = json.load(f)
#--- 
id2label = {
    0:data['categories'][0]['name'],
    1:data['categories'][1]['name'],
    2:data['categories'][2]['name'],
    3:data['categories'][3]['name'],
    4:data['categories'][4]['name']
}
label2id = {v: k for k, v in id2label.items()}

In [10]:
# define model
from models import custom_segformer
model = custom_segformer.FewShotFormer.from_pretrained("nvidia/mit-b5",
                                                         num_labels=5,
                                                         id2label=id2label,
                                                         label2id=label2id,
)

/home/eric/anaconda3/envs/trex/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of FewShotFormer were not initialized from the model checkpoint at nvidia/mit-b5 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weig

In [11]:
import torch
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import os
import evaluate
import logging
from utils import rs_utils

In [12]:
# Set up a logger
log_dir = "/home/eric/srcs/FewShotSeg_Lab/FewShotVision_Lab/Segmentation_Pipes/logs"
os.makedirs(log_dir, exist_ok=True)
log_file_path = os.path.join(log_dir, "fewshot_test_0255.log")
logger = rs_utils.setup_logger("FewShotSeg", log_file_path, level=logging.INFO)

In [13]:
# define metric
metric = evaluate.load("mean_iou")
# define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006)
# move model to GPU
device = torch.device("cuda:0")
model.to(device)
model.train()

#-----------
_EPOCHS = 50
_MODEL_SAVE = False
#----------

In [14]:
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

best_val_iou = 0 

#---
for epoch in range(_EPOCHS):  # loop over the dataset multiple times
    logger.info(f"Epoch: {epoch}")

    # Training loop
    for idx, batch in enumerate(tqdm(train_dataloader)):
        # get the inputs
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss, logits = outputs.loss, outputs.logits
        
        loss.backward()
        optimizer.step()

        # Evaluate periodically
        if idx % 100 == 0:
            with torch.no_grad():
                upsampled_logits = nn.functional.interpolate(
                    logits, size=labels.shape[-2:], mode="bilinear", align_corners=False
                )
                predicted = upsampled_logits.argmax(dim=1)

                metrics = metric._compute(
                    predictions=predicted.cpu(),
                    references=labels.cpu(),
                    num_labels=len(id2label),
                    ignore_index=255,
                    reduce_labels=False,  # we've already reduced the labels ourselves
                )
                logger.info(f"Epoch: {epoch}")
                logger.info(f"Loss: {loss.item()}")
                logger.info(f"Mean_iou: {metrics['mean_iou']}")
                logger.info(f"Mean accuracy: {metrics['mean_accuracy']}")

    # Validation loop
    val_loss = 0
    val_metrics = {"mean_iou": 0, "mean_accuracy": 0}
    for batch in tqdm(valid_dataloader):
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            outputs = model(pixel_values=pixel_values, labels=labels)
            val_loss += outputs.loss.item()

            logits = outputs.logits
            upsampled_logits = nn.functional.interpolate(
                logits, size=labels.shape[-2:], mode="bilinear", align_corners=False
            )
            predicted = upsampled_logits.argmax(dim=1)

            batch_metrics = metric._compute(
                predictions=predicted.cpu(),
                references=labels.cpu(),
                num_labels=len(id2label),
                ignore_index=255,
                reduce_labels=False,
            )
            val_metrics["mean_iou"] += batch_metrics["mean_iou"]
            val_metrics["mean_accuracy"] += batch_metrics["mean_accuracy"]
            
    # Average validation loss and metrics
    val_loss /= len(valid_dataloader)
    val_metrics["mean_iou"] /= len(valid_dataloader)
    val_metrics["mean_accuracy"] /= len(valid_dataloader)

    logger.info(f"Validation Loss: {val_loss}")
    logger.info(f"Validation Mean IoU: {val_metrics['mean_iou']}")
    logger.info(f"Validation Mean Accuracy: {val_metrics['mean_accuracy']}")

    # Save the model if the validation IoU improves
    if val_metrics["mean_iou"] > best_val_iou:
        best_val_iou = val_metrics["mean_iou"]
        
        if _MODEL_SAVE:
            torch.save(
                model.state_dict(),
                os.path.join(
                    "/disk3/eric/checkpoints/military_fewshot_seg",
                    f"segformer_best_epoch_{epoch}_miou_{best_val_iou:.4f}.pt",
                ),
            )
        logger.info("Model saved!")


2024-11-25 02:58:24 - FewShotSeg - INFO - Epoch: 0
2024-11-25 02:58:24 - FewShotSeg - INFO - Epoch: 0


  0%|          | 0/33 [00:00<?, ?it/s]

filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M2A1Slammer2__part_23.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/M2A1Slammer2__part_23.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/T140Angara__part_12.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/T140Angara__part_12.png


/home/eric/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
2024-11-25 02:58:25 - FewShotSeg - INFO - Epoch: 0
2024-11-25 02:58:25 - FewShotSeg - INFO - Epoch: 0
2024-11-25 02:58:25 - FewShotSeg - INFO - Loss: 1.6658241748809814
2024-11-25 02:58:25 - FewShotSeg - INFO - Loss: 1.6658241748809814
2024-11-25 02:58:25 - FewShotSeg - INFO - Mean_iou: 0.041045542026071836
2024-11-25 02:58:25 - FewShotSeg - INFO - Mean_iou: 0.041045542026071836
2024-11-25 02:58:25 - FewShotSeg - INFO - Mean accuracy: 0.5224907644595015
2024-11-25 02:58:25 - FewShotSeg - INFO - Mean accuracy: 0.5224907644595015


filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M5Sandstorm__part_18.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/M5Sandstorm__part_18.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M5Sandstorm__part_14.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/M5Sandstorm__part_14.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/Zamak__part_18.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/Zamak__part_18.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/T140Angara__part_6.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/T140Angara__part_6.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M2A1Slammer2__part_14.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentatio

...

2024-11-25 03:06:02 - FewShotSeg - INFO - Epoch: 49
2024-11-25 03:06:02 - FewShotSeg - INFO - Epoch: 49
2024-11-25 03:06:02 - FewShotSeg - INFO - Loss: 0.0029899771325290203
2024-11-25 03:06:02 - FewShotSeg - INFO - Loss: 0.0029899771325290203
2024-11-25 03:06:02 - FewShotSeg - INFO - Mean_iou: 0.9432405240172611
2024-11-25 03:06:02 - FewShotSeg - INFO - Mean_iou: 0.9432405240172611
2024-11-25 03:06:02 - FewShotSeg - INFO - Mean accuracy: 0.9777461315165846
2024-11-25 03:06:02 - FewShotSeg - INFO - Mean accuracy: 0.9777461315165846


filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M5Sandstorm__part_19.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/M5Sandstorm__part_19.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M5Sandstorm__part_16.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/M5Sandstorm__part_16.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/Zamak__part_17.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/Zamak__part_17.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/T140Angara__part_4.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/labels/T140Angara__part_4.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/train/images/M5Sandstorm__part_11.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation

  0%|          | 0/8 [00:00<?, ?it/s]

filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/valid/images/M2A1Slammer2__part_10.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/valid/labels/M2A1Slammer2__part_10.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/valid/images/M2A1Slammer2__part_13.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/valid/labels/M2A1Slammer2__part_13.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/valid/images/M2A1Slammer2__part_16.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/valid/labels/M2A1Slammer2__part_16.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/valid/images/M2A1Slammer2__part_18.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/valid/labels/M2A1Slammer2__part_18.png
filename /disk3/eric/dataset/VISION_SOFS/WEAPON_5/segmentation_pipe/valid/images/M2A1Slammer2__part_20.png
filename /disk3/eric/dataset/VISION_S

2024-11-25 03:06:11 - FewShotSeg - INFO - Validation Loss: 0.005245164065854624
2024-11-25 03:06:11 - FewShotSeg - INFO - Validation Loss: 0.005245164065854624
2024-11-25 03:06:11 - FewShotSeg - INFO - Validation Mean IoU: 0.7221647160490048
2024-11-25 03:06:11 - FewShotSeg - INFO - Validation Mean IoU: 0.7221647160490048
2024-11-25 03:06:11 - FewShotSeg - INFO - Validation Mean Accuracy: 0.9079027870596078
2024-11-25 03:06:11 - FewShotSeg - INFO - Validation Mean Accuracy: 0.9079027870596078
